<h1 style="text-align: center">
Foundations of DataScience </br>
</h1>
<h2 style="text-align: center">
Course Project </br>
Financial Analysis on Twitter
</h2>




  <style>
    table {
      width: 100%;
      border-collapse: collapse;
    }

    th, td {
      border: 1px solid #dddddd;
      text-align: left;
      padding: 8px;
    }

    th {
      background-color: #f2f2f2;
    }

    .box {
      border: 1px solid #000;
      padding: 10px;
      width: 400px; /* Adjust the width as needed */
      margin: 20px auto;
    }
  </style>
</head>
<body>

<div class="box">
  <table>
    <tr>
      <th colspan="2">Personal Info</th>
    </tr>
    <tr>
      <td>First Name:</td>
      <td>Ali</td>
    </tr>
    <tr>
      <td>Last Name:</td>
      <td>Nikkhah</td>
    </tr>
    <tr>
      <td>Student Number:</td>
      <td>99102445</td>
    </tr>
    <tr>
      <td>Git:</td>
      <td><a href="https://github.com/AliNikkhah2001/DataScience02" target="_blank">https://github.com/AliNikkhah2001/DataScience02</a></td>
    </tr>
  </table>
</div>



<!DOCTYPE html>

<body>

<h2>Phase I</h2>

<table border="1">
  <tr>
    <th>Task</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Load dataset and preprocess</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Express information of least and most tweeted stocks by segmentation over companies based on tweets related to them</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Distribution of 5 stocks over time</td>
  </tr>
  <tr>
    <td>4</td>
    <td>Distribution of all financial tweets over time</td>
  </tr>
  <tr>
    <td>5</td>
    <td>Distribution of retweets per tweets that company mentioned on</td>
  </tr>
  <tr>
    <td>6</td>
    <td>Most information of 2 stocks computed solely from financial dataset</td>
  </tr>
  <tr>
    <td>7</td>
    <td>Movement directions of two stocks compared to real-world news</td>
  </tr>
  <tr>
    <td>8</td>
    <td>Co-occurrence of stocks over tweets</td>
  </tr>
</table>

</body>
</html>


* Libraries installation and initialization

In [ ]:
%%capture
!pip install --upgrade pip

In [ ]:
%%capture
# Install necessary libraries
!pip install tensorflow-data-validation

In [ ]:
%%capture
import zipfile
import urllib.request
import os
import pandas as pd
import pandas as pd
import tensorflow_data_validation as tfdv
from tensorflow_data_validation.utils import display_util
from tqdm import tqdm
import os
import urllib.request
import zipfile


* Load dataset

In [ ]:
# Create a folder for the dataset if it doesn't exist
dataset_folder = 'dataset/P1'
os.makedirs(dataset_folder, exist_ok=True)

# List of dataset URLs
dataset_urls = [
    'https://zenodo.org/records/2686862/files/companies.csv.zip?download=1',
    'https://zenodo.org/records/2686862/files/entities.csv.zip?download=1',
    'https://zenodo.org/records/2686862/files/tweets.csv.zip?download=1',
    'https://zenodo.org/records/2686862/files/users.csv.zip?download=1'
]

# Download and extract each dataset with tqdm progress bar
for url in tqdm(dataset_urls, desc="Downloading and extracting datasets"):
    file_name = url.split('/')[-1].split('?')[0]
    zip_file_path = os.path.join(dataset_folder, file_name)
    csv_file_path = os.path.join(dataset_folder, file_name.replace('.zip', ''))

    # Download the zip file
    urllib.request.urlretrieve(url, zip_file_path)

    # Extract the contents of the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(csv_file_path)

    # Remove the zip file after extraction
    os.remove(zip_file_path)


In [ ]:
dataset_files = ['companies.csv/companies', 'entities.csv/entities', 'tweets.csv/tweets']

# Create a list to store the DataFrames
dataframes = []

# Use tqdm to show progress bar while loading datasets
for file in tqdm(dataset_files, desc="Loading Datasets", unit="file"):
    file_path = os.path.join(dataset_folder, file + '.csv')
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Unpack the DataFrames
companies_df, entities_df, tweets_df = dataframes

Loading Datasets: 100%|██████████| 3/3 [01:03<00:00, 21.06s/file]


**We face several erros on parsing users dataframe. in the comming cells of the notebook, we try to solve this problem**

In [ ]:
%%capture
# Load 'users.csv' into a pandas DataFrame, skipping lines with extra fields
users_df_path = os.path.join(dataset_folder, 'users.csv/users.csv')
users_df = pd.read_csv(users_df_path, error_bad_lines=False)

In [ ]:
print("\nInformation about the Users DataFrame:")
print(users_df.info())

print("\nInformation about the Companies DataFrame:")
print(companies_df.info())

print("\nInformation about the Entities DataFrame:")
print(entities_df.info())

print("\nInformation about the Tweets DataFrame:")
print(tweets_df.info())


Information about the Users DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597411 entries, 0 to 597410
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                597411 non-null  int64  
 1   name              593020 non-null  object 
 2   screen_name       597411 non-null  object 
 3   statuses_count    597411 non-null  int64  
 4   followers_count   597411 non-null  int64  
 5   friends_count     597411 non-null  int64  
 6   favourites_count  597411 non-null  int64  
 7   listed_count      597407 non-null  object 
 8   url               150160 non-null  object 
 9   lang              597407 non-null  object 
 10  time_zone         243540 non-null  object 
 11  location          333734 non-null  object 
 12  verified          4591 non-null    object 
 13  description       409968 non-null  object 
 14  created_at        597405 non-null  object 
 15  bot               25988 non-

In [ ]:

def remove_zero_columns_rows(df):
    # Remove zero columns
    zero_columns_before = df.shape[1]
    df = df.loc[:, (df != 0).any(axis=0)]
    zero_columns_after = df.shape[1]

    # Remove zero rows
    zero_rows_before = df.shape[0]
    df = df.loc[(df != 0).any(axis=1)]
    zero_rows_after = df.shape[0]

    # Print the number of zero rows and columns deleted
    print(f"Number of zero columns deleted: {zero_columns_before - zero_columns_after}")
    print(f"Number of zero rows deleted: {zero_rows_before - zero_rows_after}")

    return df
users_df = remove_zero_columns_rows(users_df)
companies_df = remove_zero_columns_rows(companies_df)
entities_df = remove_zero_columns_rows(entities_df)
tweets_df = remove_zero_columns_rows(tweets_df)


Number of zero columns deleted: 0
Number of zero rows deleted: 0
Number of zero columns deleted: 0
Number of zero rows deleted: 0
Number of zero columns deleted: 0
Number of zero rows deleted: 0
Number of zero columns deleted: 0
Number of zero rows deleted: 0


In [ ]:

def remove_duplicates(df):
    # Count the number of duplicated rows before removal
    duplicates_before = df.duplicated().sum()

    # Remove duplicated rows
    df = df.drop_duplicates()

    # Count the number of duplicated rows after removal
    duplicates_after = df.duplicated().sum()

    # Print the number of removed duplicate rows
    print(f"Number of duplicated rows removed: {duplicates_before - duplicates_after}")

    return df

# Apply the function to each DataFrame with tqdm progress bar
users_df = remove_duplicates(users_df)
companies_df = remove_duplicates(companies_df)
entities_df = remove_duplicates(entities_df)
tweets_df = remove_duplicates(tweets_df)


Number of duplicated rows removed: 0
Number of duplicated rows removed: 0
Number of duplicated rows removed: 0
Number of duplicated rows removed: 0


 * Statistics on most and least tweeted stocks. Perform segmentation of the companies based on the number of tweets they have. Provide relevant visualizations.

In [ ]:
print("Companies DataFrame:")
random_company_sample = companies_df.sample(n=1)
print(random_company_sample.to_string(index=False))

# Print one random sample from tweets_df
print("\nTweets DataFrame:")
random_tweet_sample = tweets_df.sample(n=1)
print(random_tweet_sample.to_string(index=False))

# Print one random sample from entities_df
print("\nEntities DataFrame:")
random_entity_sample = entities_df.sample(n=1)
print(random_entity_sample.to_string(index=False))

Companies DataFrame:
ticker                 name exchange  capitalization
  BMTC Bryn Mawr Bank Corp.   NASDAQ     730480000.0

Tweets DataFrame:
                id                                                                                                    text            user_id  in_reply_to_status_id  in_reply_to_user_id  retweeted_status_id  retweeted_user_id lang                                                        source                     created_at
866962922362126336 https://t.co/KA6yh5M9go Trading Expert Jason Bond has a Special Offer Just for You! $COP $VLO $APC $APA 726445144812081154                      0                    0                    0                  0   en <a href="https://www.smqueue.com/" rel="nofollow">smqueue</a> Tue May 23 10:24:02 +0000 2017

Entities DataFrame:
          tweet_id entity_type           text
895697443098165248     mention NEO_Blockchain


In [ ]:
companies_tickers = companies_df['ticker'].to_list()

In [ ]:
import pandas as pd
import numpy as np

# Assuming tweets_df is the DataFrame containing the tweets and companies_df is the DataFrame containing companies information

# Create a new column 'tickers_list' in the Tweets DataFrame to store the found tickers
tweets_df['tickers_list'] = np.nan

# Create an empty list to store found tickers for each row
found_tickers_list = []

# Loop through each row in the tweets DataFrame and search for company tickers
for index, row in tweets_df.iterrows():
    tweet_text = row['text']
    found_tickers = []

    # Loop through each ticker in the Companies DataFrame
    for company_ticker in companies_df['ticker']:
        # Check if the ticker exists in the tweet text
        if str(company_ticker) in str(tweet_text):
            found_tickers.append(company_ticker)

    # Append the list of found tickers for this row to the overall list
    found_tickers_list.append(found_tickers)

# Assign the list of found tickers to the 'tickers_list' column in the Tweets DataFrame
tweets_df['tickers_list'] = found_tickers_list

# Display the modified Tweets DataFrame with the 'tickers_list' column
print(tweets_df.head())


In [ ]:
print(len(companies_tickers))

30032


In [ ]:
print(tweets_df)

                         id  \
0        865326129644797957   
1        865326133008642049   
2        865326134262681600   
3        865326134917050368   
4        865326135952855040   
...                     ...   
9091534  907725480618610689   
9091535  907725484909309957   
9091536  907725487996272640   
9091537  907725490760376320   
9091538  907725492874223616   

                                                      text  \
0        RT @cruzfloresiv: To all the weak hands who so...   
1        Investors Eye Fed, But Bond ETFs Still Add Ass...   
2        Sell $NLNK (Ne❑wLink Genetics Corporation) tha...   
3        Increase: $VCO $NTES $BPT $ENIC $QIWI $JP $STX...   
4        Former #FDA commissioner Califf joins Verily, ...   
...                                                    ...   
9091534  Contrasting Pure Cycle Corporation $PCYO &amp;...   
9091535  Contrasting Harvard Bioscience $HBIO &amp; Ful...   
9091536  NRG Energy, Inc. $NRG Receives $21.13 Average ...   
9091537

In [ ]:
print(tweets_df['text'][12])

https://t.co/0iuk9iq2il // Video Analysis $AMAT // $AMAT #trading #investing #stocks https://t.co/10dHVbxARc


• Statistics on distributions of 5 individual stocks over time. Choose the individual stocks to perform reflect different sectors of the economy.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming tweets_df contains the relevant information about tweets

# Choose five individual stocks representing different sectors
selected_stocks = ['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'TSLA']

# Filter the dataset for the selected stocks
selected_stocks_df = tweets_df[tweets_df['ticker'].isin(selected_stocks)]

# Convert the timestamp column to datetime format
selected_stocks_df['created_at'] = pd.to_datetime(selected_stocks_df['created_at'])

# Set the timestamp column as the index
selected_stocks_df.set_index('created_at', inplace=True)

# Plot the distribution of tweets over time for each selected stock
plt.figure(figsize=(15, 8))
for stock in selected_stocks:
    stock_data = selected_stocks_df[selected_stocks_df['ticker'] == stock]
    stock_data.resample('M').size().plot(label=stock, linewidth=2)

plt.title('Distribution of Tweets Over Time for Selected Stocks')
plt.xlabel('Date')
plt.ylabel('Number of Tweets')
plt.legend()
plt.show()



  <style>
    body {
      font-family: sans-serif;
      margin: 20px;
    }

    h1, h2 {
      color: #333;
    }

    p {
      color: #666;
    }

    .code {
      font-family: 'Courier New', monospace;
      background-color: #f4f4f4;
      padding: 10px;
      border-radius: 5px;
    }
  </style>
</head>
<body>

<h1>Part 2. Sentiment Analysis</h1>

<h2>Loading and Preprocessing the Dataset</h2>
<p>Let's start by loading and preprocessing the dataset. initial steps to prepare the data for exploratory analysis.</p>




* Libraries installation and initialization

In [ ]:
!pip install --upgrade pip

In [ ]:
# Install necessary libraries
!pip install tqdm pandas matplotlib tensorflow-data-validation

In [ ]:
!git clone https://github.com/tensorflow/data-validation
!cd data-validation

In [ ]:
!python3 setup.py bdist_wheel

In [ ]:
pip install dist/*.whl

In [ ]:
# Import libraries
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
#import tensorflow_data_validation as tfdv

In [ ]:

file_path = 'training.1600000.processed.noemoticon.csv'
# Load the dataset with tqdm progress bar
tqdm.pandas()
df = pd.read_csv(file_path, header=None, encoding='latin1')



In [ ]:
# Display basic information about the dataset
print("Head of the DataFrame:")
print(df.head())

print("\nInformation about the DataFrame:")
print(df.info())



In [ ]:

# Plot sentiment distribution in a pie plot
sentiment_counts = df[0].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=90, colors=['red', 'gray', 'green'])
plt.title('Sentiment Distribution')
plt.show()


In [ ]:

# Plot tweet length distribution with higher bins
plt.figure(figsize=(12, 6))
plt.hist(df[5].apply(len), bins=50, color='skyblue')
plt.title('Tweet Length Distribution')
plt.xlabel('Tweet Length')
plt.ylabel('Count')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming df is your DataFrame
# If not, replace df with your actual DataFrame

# Filter negative and positive sentiment tweets
negative_tweets = df[df[0] == 0]
positive_tweets = df[df[0] == 4]

# Convert date columns to datetime
negative_tweets[2] = pd.to_datetime(negative_tweets[2].astype(str).str.replace('PDT', ''), errors='coerce').dt.tz_localize('UTC')
positive_tweets[2] = pd.to_datetime(positive_tweets[2].astype(str).str.replace('PDT', ''), errors='coerce').dt.tz_localize('UTC')

# Plot negative sentiment tweets over time
plt.figure(figsize=(12, 6))
negative_tweets.groupby(negative_tweets[2].dt.date).size().plot(marker='o', linestyle='-', color='red', label='Negative Sentiment')
positive_tweets.groupby(positive_tweets[2].dt.date).size().plot(marker='o', linestyle='-', color='green', label='Positive Sentiment')
plt.title(' Sentiment Tweets Over Time')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.show()


In [ ]:

num_unique_users = df[4].nunique()
print(f"Number of Unique User IDs: {num_unique_users}")

### Information about the DataFrame:

The dataset contains **1,600,000** entries with **6** columns. Here's an explanation of each column:

1. **Column 0 (0):**
   - **Data Type:** int64
   - **Non-Null Count:** 1,600,000
   - This column appears to be of numeric type (`int64`) and has no missing values. It might represent some numerical information.

2. **Column 1 (1):**
   - **Data Type:** int64
   - **Non-Null Count:** 1,600,000
   - Similar to Column 0, this column is also of numeric type (`int64`) with no missing values.

3. **Column 2 (2):**
   - **Data Type:** object
   - **Non-Null Count:** 1,600,000
   - This column is of object type, typically indicating text or categorical data.

4. **Column 3 (3):**
   - **Data Type:** object
   - **Non-Null Count:** 1,600,000
   - Another object-type column, likely containing textual or categorical information.

5. **Column 4 (4):**
   - **Data Type:** object
   - **Non-Null Count:** 1,600,000
   - Similar to Columns 2 and 3, this column is of object type.

6. **Column 5 (5):**
   - **Data Type:** object
   - **Non-Null Count:** 1,600,000
   - The last column is also of object type, indicating text or categorical data.

The overall memory usage for the DataFrame is **73.2+ MB**.

This information provides an overview of the data types, non-null counts, and memory usage for each column in the DataFrame.


<style>
  table {
    font-family: Arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
  }

  th, td {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
  }

  th {
    background-color: #f2f2f2;
  }
</style>
</head>
<body>

<h2>References</h2>

<table>
  <tr>
    <th>Reference</th>
    <th>Link</th>
  </tr>
  <tr>
    <td>Reference 1</td>
    <td><a href="https://arxiv.org/pdf/1804.04406.pdf" target="_blank">https://arxiv.org/pdf/1804.04406.pdf</a></td>
  </tr>
  <!-- Add more rows for additional references if needed -->
</table>